In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

pd.set_option('display.max_columns', 30)

In [2]:
# import trip data
df = pd.read_csv('data/citibike_clean.csv')
df.shape

(1863447, 17)

In [4]:
# import zipcodes shapefile 
zipcodes = gpd.read_file('nyc_zip/ZIP_CODE_040114.shp')[['ZIPCODE', 'geometry']]

In [3]:
# convert trip originating stations to geo dataframe
start_gpd = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(
    df['start_lon'], df['start_lat']), crs = 'epsg:4326')

# update projection 
start_gpd.to_crs(epsg=2263, inplace = True)

In [5]:
# spatial join to match each trip start with its zipcode
trip_start = start_gpd.sjoin(zipcodes, how='left')
trip_start.head()

,tripduration,starttime,stoptime,start_id,start_name,start_lat,start_lon,end_id,end_name,end_lat,end_lon,bikeid,usertype,birth_year,gender,weekday,hour,geometry,index_right,ZIPCODE
0,17.700000,2020-06-01 00:00:03.372,2020-06-01 00:17:46.208,3419,Douglass St & 4 Ave,40.679279,-73.981540,3419,Douglass St & 4 Ave,40.679279,-73.981540,39852,Customer,1997,2,0,0,POINT (989370.184 186756.539),116.0,11217
1,63.500000,2020-06-01 00:00:03.553,2020-06-01 01:03:33.936,366,Clinton Ave & Myrtle Ave,40.693261,-73.968896,336,Sullivan St & Washington Sq,40.730477,-73.999061,37558,Subscriber,1969,0,0,0,POINT (992875.414 191851.653),110.0,11205
2,16.950000,2020-06-01 00:00:09.614,2020-06-01 00:17:06.833,389,Broadway & Berry St,40.710446,-73.965251,3562,Classon Ave & St Marks Ave,40.676520,-73.959608,37512,Customer,1988,2,0,0,POINT (993883.824 198112.873),260.0,11249
3,3.766667,2020-06-01 00:00:12.178,2020-06-01 00:03:58.864,3255,8 Ave & W 31 St,40.750585,-73.994685,505,6 Ave & W 33 St,40.749013,-73.988484,39674,Customer,1969,0,0,0,POINT (985722.681 212735.192),82.0,10001
4,23.950000,2020-06-01 00:00:21.255,2020-06-01 00:24:18.965,367,E 53 St & Lexington Ave,40.758281,-73.970694,497,E 17 St & Broadway,40.737050,-73.990093,21093,Customer,1997,2,0,0,POINT (992368.802 215540.154),193.0,10022


In [7]:
# identify unmatched stations
zip_null = trip_start[trip_start['ZIPCODE'].isna()]
zip_null['start_id'].value_counts()

315     4478
3609       1
514        1
Name: start_id, dtype: int64

In [6]:
# update missing zipcodes
trip_start.loc[trip_start.start_id == 315, 'ZIPCODE'] = 10043
trip_start.loc[trip_start.start_id == 3609, 'ZIPCODE'] = 11102
trip_start.loc[trip_start.start_id == 514, 'ZIPCODE'] = 10018

trip_start['ZIPCODE'].isnull().sum()

0

In [8]:
# convert trip destination stations to geo dataframe
end_gpd = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(
    df['end_lon'], df['end_lat']), crs = 'epsg:4326')

# update projection 
end_gpd.to_crs(epsg=2263, inplace = True)

In [9]:
# spatial join to match each trip end with its zipcode
trip_end = end_gpd.sjoin(zipcodes, how='left')
trip_end.head()

,tripduration,starttime,stoptime,start_id,start_name,start_lat,start_lon,end_id,end_name,end_lat,end_lon,bikeid,usertype,birth_year,gender,weekday,hour,geometry,index_right,ZIPCODE
0,17.700000,2020-06-01 00:00:03.372,2020-06-01 00:17:46.208,3419,Douglass St & 4 Ave,40.679279,-73.981540,3419,Douglass St & 4 Ave,40.679279,-73.981540,39852,Customer,1997,2,0,0,POINT (989370.184 186756.539),116.0,11217
1,63.500000,2020-06-01 00:00:03.553,2020-06-01 01:03:33.936,366,Clinton Ave & Myrtle Ave,40.693261,-73.968896,336,Sullivan St & Washington Sq,40.730477,-73.999061,37558,Subscriber,1969,0,0,0,POINT (984510.345 205409.211),95.0,10012
2,16.950000,2020-06-01 00:00:09.614,2020-06-01 00:17:06.833,389,Broadway & Berry St,40.710446,-73.965251,3562,Classon Ave & St Marks Ave,40.676520,-73.959608,37512,Customer,1988,2,0,0,POINT (995453.816 185753.400),117.0,11238
3,3.766667,2020-06-01 00:00:12.178,2020-06-01 00:03:58.864,3255,8 Ave & W 31 St,40.750585,-73.994685,505,6 Ave & W 33 St,40.749013,-73.988484,39674,Customer,1969,0,0,0,POINT (987440.832 212162.395),82.0,10001
4,23.950000,2020-06-01 00:00:21.255,2020-06-01 00:24:18.965,367,E 53 St & Lexington Ave,40.758281,-73.970694,497,E 17 St & Broadway,40.737050,-73.990093,21093,Customer,1997,2,0,0,POINT (986995.506 207803.886),90.0,10003


In [10]:
zip_null = trip_end[trip_end['ZIPCODE'].isna()]
zip_null['end_id'].value_counts()

315     4365
3199       5
3791       3
3276       2
3277       1
3184       1
3639       1
3609       1
3198       1
3203       1
514        1
3792       1
3211       1
Name: end_id, dtype: int64

In [10]:
# update missing NYC zipcodes

trip_end.loc[trip_end.end_id == 315, 'ZIPCODE'] = 10043
trip_end.loc[trip_end.end_id == 3609, 'ZIPCODE'] = 11102
trip_end.loc[trip_end.end_id == 514, 'ZIPCODE'] = 10018

# remaining missing ZIPs are in New Jersey
trip_end['ZIPCODE'].isnull().sum()

17

In [17]:
# add start and end zip to one dataframe
trip_start.rename(columns={'ZIPCODE': 'zip_start'}, inplace = True)
trips = trip_start
trips['zip_end'] = trip_end['ZIPCODE']
trips.head()

,tripduration,starttime,stoptime,start_id,start_name,start_lat,start_lon,end_id,end_name,end_lat,end_lon,bikeid,usertype,birth_year,gender,weekday,hour,geometry,zip_start,zip_end
0,17.700000,2020-06-01 00:00:03.372,2020-06-01 00:17:46.208,3419,Douglass St & 4 Ave,40.679279,-73.981540,3419,Douglass St & 4 Ave,40.679279,-73.981540,39852,Customer,1997,2,0,0,POINT (989370.184 186756.539),11217,11217
1,63.500000,2020-06-01 00:00:03.553,2020-06-01 01:03:33.936,366,Clinton Ave & Myrtle Ave,40.693261,-73.968896,336,Sullivan St & Washington Sq,40.730477,-73.999061,37558,Subscriber,1969,0,0,0,POINT (992875.414 191851.653),11205,10012
2,16.950000,2020-06-01 00:00:09.614,2020-06-01 00:17:06.833,389,Broadway & Berry St,40.710446,-73.965251,3562,Classon Ave & St Marks Ave,40.676520,-73.959608,37512,Customer,1988,2,0,0,POINT (993883.824 198112.873),11249,11238
3,3.766667,2020-06-01 00:00:12.178,2020-06-01 00:03:58.864,3255,8 Ave & W 31 St,40.750585,-73.994685,505,6 Ave & W 33 St,40.749013,-73.988484,39674,Customer,1969,0,0,0,POINT (985722.681 212735.192),10001,10001
4,23.950000,2020-06-01 00:00:21.255,2020-06-01 00:24:18.965,367,E 53 St & Lexington Ave,40.758281,-73.970694,497,E 17 St & Broadway,40.737050,-73.990093,21093,Customer,1997,2,0,0,POINT (992368.802 215540.154),10022,10003


In [18]:
# export to csv with both start and end zip
trip_start.to_csv('data/citibike_ZIP.csv', index = False)